In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# library for data analysis
import pandas as pd
# library for file explorer
import os
PATH = os.getcwd()
print(PATH)
# library for file management
import glob
# library for data science
import numpy as np
# library for Date Time format
import datetime

# display info options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  # default='warn'

# Import Standard Logs and clean data

In [ ]:
foldername = "Esempio 2/"

# Define files path
Path = "S?*.csv"

# Read all the files name in the directory
AllFilesname = glob.glob(foldername + Path)

# Create an empty list to store the dataframes
ListDataframe = list()

# Total file number
NoFiles = len(AllFilesname)

# For each file in the specified directory import the data and add it in the list
for Filename in AllFilesname:
    ListDataframe.append(pd.read_csv(Filename, sep=';', skiprows=3, decimal=',', encoding='unicode_escape'))

# Concatenate the files in the list in unique dataframe
DF_Data = pd.concat(ListDataframe, axis=0, ignore_index=True)

# Drop columns unused
DF_Data.drop(['GpsPos','Unnamed: 45'], axis=1, inplace=True)

# DateTime
DF_Data['DateTime'] = pd.to_datetime(DF_Data['DateTime'], format="%Y-%m-%d %H:%M:%S")
# ordering ascending
DF_Data = DF_Data.sort_values(by='DateTime', ascending=True)

# Reorder Columns
DF_Data= DF_Data[['DateTime', 'ChangeOverInProgress','F60InAutoMode','CV1_Position', 'CV2_Position',
                  'CV3_Position', 'CV4_Position', 'CV5_Position', 'SupplyCurrentPump',
       'CircCurrentPump', 'CurrentControl', 'CurrentFilter',
       'PT1', 'PT2', 'TT1', 'TT2', 'TargetTemperature',
       'TemperatureLowLimit', 'TemperatureHighLimit', 'VT', 'TargetViscosity',
       'ViscosityLowLimit', 'ViscosityHighLimit', 'Density', 'FM1_MassFlow',
       'FM1_Density', 'FM1_Temperature', 'FM2_MassFlow', 'FM2_Density',
       'FM2_Temperature', 'FM3_MassFlow', 'FM3_Density', 'FM3_Temperature',
       'FM4_MassFlow', 'FM4_Density', 'FM4_Temperature', 'FT_VolumeFlow',
       'FT_MassFlow', 'FT_Density', 'FT_Temperature', 'SO2', 'CO2', 'SC','DPT_AI']]

# Columns as float
DF_Data.iloc[:,12:] = DF_Data.iloc[:,12:].astype(float)

#Remove duplicates
DF_Data.drop_duplicates(keep=False, inplace=True)

# Show results
print(DF_Data.shape)
DF_Data.head()

In [ ]:
copia = DF_Data[['DateTime', 'PT1']]
copia.head()

# Alarms and Events Import

In [ ]:
foldername = "Esempio 2/"

# Define files path
Path = "A?*.csv"

# Read all the files name in the directory
AllFilesname = glob.glob(foldername + Path)

# Create an empty list to store the dataframes
ListDataframe = list()

# Total file number
NoFiles = len(AllFilesname)

# For each file in the specified directory import the data and add it in the list
for Filename in AllFilesname:
    ListDataframe.append(pd.read_csv(Filename, sep=';', skiprows=3, decimal=',', encoding='unicode_escape'))

# Concatenate the files in the list in unique dataframe
Alarms = pd.concat(ListDataframe, axis=0, ignore_index=True)

Alarms.drop(['Unnamed: 2'], axis=1, inplace=True)

# DateTime
Alarms['DateTime'] = pd.to_datetime(Alarms['DateTime'], format="%Y-%m-%d %H:%M:%S")
# ordering ascending
Alarms = Alarms.sort_values(by='DateTime', ascending=True)

#Remove duplicates
Alarms.drop_duplicates(keep=False, inplace=True)

# Show results
print(Alarms.shape)
Alarms.head()

In [ ]:
foldername = "Esempio 2/"

# Define files path
Path = "E?*.csv"

# Read all the files name in the directory
AllFilesname = glob.glob(foldername + Path)

# Create an empty list to store the dataframes
ListDataframe = list()

# Total file number
NoFiles = len(AllFilesname)

# For each file in the specified directory import the data and add it in the list
for Filename in AllFilesname:
    ListDataframe.append(pd.read_csv(Filename, sep=';', skiprows=3, decimal=',', encoding='unicode_escape'))

# Concatenate the files in the list in unique dataframe
Events = pd.concat(ListDataframe, axis=0, ignore_index=True)

Events.drop(['GpsPos', 'Unnamed: 4', 'Data'], axis=1, inplace=True)

# DateTime
Events['DateTime'] = pd.to_datetime(Events['DateTime'], format="%Y-%m-%d %H:%M:%S")
# ordering ascending
Events = Events.sort_values(by='DateTime', ascending=True)

#Remove duplicates
Events.drop_duplicates(keep=False, inplace=True)

# Show results
print(Events.shape)
Events.head()

In [ ]:
DF_Data.dtypes

In [ ]:
#print(DF_Data[(DF_Data['ChangeOverInProgress'] == 1)]['DateTime'].min())
#print(DF_Data[(DF_Data['ChangeOverInProgress'] == 1)]['DateTime'].max())

# Change Overs

In [ ]:
# Modify wrong cell to simulate Changeover
DF_Data.at[2498,'ChangeOverInProgress'] = 1

# Identify when Change Over Started and Finished
# ChangeOverInProgress = O : no change
# ChangeOverInProgress = 1 : started
# ChangeOverInProgress = -1 : finished

# Create column with value change
DF_Data['ChangeoverCMDchange'] = DF_Data['ChangeOverInProgress'].diff()
# Show results
DF_Data[(DF_Data['ChangeoverCMDchange'] != 0)]

In [ ]:
# Calculate number of changeovers in dataset loaded
NoCO = DF_Data[(DF_Data['ChangeoverCMDchange'] == 1)].shape[0]
print(NoCO)

# memorize starting DateTimes
COstart = DF_Data[(DF_Data['ChangeoverCMDchange'] == 1)]['DateTime'].values
print(COstart)

# memorize finishing DateTimes
COfinish = DF_Data[(DF_Data['ChangeoverCMDchange'] == -1)]['DateTime'].values
print(COfinish)

In [ ]:
# Delta time before and after change over
delta = np.timedelta64(20,'m') # 20 minutes

# Create n dataframes for each changeover +- 20 minutes
COs = []
for i in range(NoCO):
    df = DF_Data[(DF_Data['DateTime'] >= COstart[i]-delta) & (DF_Data['DateTime'] <= COfinish[i]+delta)]
    COs.append(df)

In [ ]:
# declare columns to show
# ChOvColumns = ['DateTime','CV1_Position','CV4_Position','TT2','TargetTemperature','VT', 'ChangeOverInProgress']

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go

config = {'displaylogo': False}

i = 1
for df in COs:
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TT2'],
        name='Actual Temperature'
    ))
    
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['VT'],
        name='Viscosity',
        yaxis="y2"))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TargetTemperature'],
        name='Target Temperature'
    ))
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV1_Position'],
        name="CV1 Position",
        yaxis="y3"
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV4_Position'],
        name="CV4 Position",
        yaxis="y3"
    ))
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ChangeOverInProgress'],
        name="Change Over InProgress",
        yaxis="y4"
    ))

    # Create axis objects
    fig.update_layout(
        xaxis=dict(domain=[0.1, 0.9]), #compress x axis 10% left an right

        yaxis=dict(
            title="Temperature"),
        yaxis2=dict(
            title="Viscosity",
            anchor="free",
            overlaying="y",
            side="left",
            position=0
        ),
        yaxis3=dict(
            title="CV Position",
            anchor="x",
            overlaying="y",
            side="right"
        ),
        yaxis4=dict(
            title="Change Over",
            anchor="free",
            overlaying="y",
            side="right",
            position=1
        ),
        legend=dict(orientation="v", x = 1.1, borderwidth= 1)
    )
    

    # Update layout properties
    fig.update_layout(
        title_text=("ChangeOver " + str(i))
    )

    fig.show()
    fig.write_html("ChangeOver_" + str(i) + ".html",
                   config=config)
    i = i+1

# Viscosity and Temperature trend

In [ ]:
# Merge Standard logs with Events and Alarms
# Merge all DataFrames
mergeDF2 = pd.merge(Alarms, Events, on='DateTime', how='outer')
mergeDF = pd.merge(DF_Data, mergeDF2, on='DateTime', how='outer')

# Sort again
mergeDF = mergeDF.sort_values(by='DateTime', ascending=True)

# show results
print (mergeDF.shape)
mergeDF.head()

In [ ]:
# Generate limits for intervals of 5 days
mergeDF['DateTime'] = pd.to_datetime(mergeDF['DateTime'])

mindate = mergeDF['DateTime'].min()
maxdate = mergeDF['DateTime'].max()
print(mindate,maxdate)

step = 5

limits = []
limits.append(mindate.date()-datetime.timedelta(days=1))

interval = maxdate - mindate

while interval > datetime.timedelta(days=step):
    limits.append(limits[-1] + datetime.timedelta(days=step))
    interval = interval - datetime.timedelta(days=step)
    
limits.append(maxdate.date()+datetime.timedelta(days=1))

limits = np.array(limits, dtype='datetime64')

# Show Results

for item in limits:
    print (item)

In [ ]:
# Divide all data in slices of 5 days
dfs = []
totalshape = 0

for i in range(len(limits)-1):
    df = pd.DataFrame()
    df = mergeDF[(mergeDF['DateTime'] > limits[i]) & (mergeDF['DateTime'] <= limits[i+1])]
    dfs.append(df)
    
    totalshape = totalshape + df.shape[0]
    print(df.shape)
    
print (totalshape)

In [ ]:
# SPECIFIC FOR THIS DATA SET!!!!!!

# select only the last 6 Dataframes!!
dfs = dfs[-6:]

# show results
for df in dfs:
    print(df.shape)

In [ ]:
# VISCOSITY

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go

config = {'displaylogo': False}

for df in dfs:
    
    df = df[['DateTime','VT', 'TargetViscosity','ViscosityLowLimit', 'ViscosityHighLimit', 'CV1_Position']]
    df = df.dropna(axis=0, how='all')
    if df.empty:
        continue
    
    mindate = df['DateTime'].min().strftime('%Y-%m-%d')
    maxdate = df['DateTime'].max().strftime('%Y-%m-%d')
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['VT'],
        name='Actual Viscosity'
    ))
    
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TargetViscosity'],
        name='Target Viscosity'
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ViscosityLowLimit'],
        name='Viscosity Low Limit'
    ))
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ViscosityHighLimit'],
        name='Viscosity High Limit'
    ))
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV1_Position'],
        name="CV1 Position",
        yaxis="y2"
    ))

    # Create axis objects
    fig.update_layout(
        xaxis=dict(domain=[0.1, 1]), #compress x axis 10% left

        yaxis=dict(
            title="Viscosity"),
        yaxis2=dict(
            title="CV Position",
            anchor="x",
            overlaying="y",
            side="right"
        ),
        legend=dict(orientation="v", x = 1.1, borderwidth= 1)
    )
    

    # Update layout properties
    fig.update_layout(
        title_text=("Viscosity " + mindate + " " + maxdate)
    )

    fig.show()
    fig.write_html("Viscosity_" + mindate + "_" + maxdate + ".html",
                   config=config)


In [ ]:
# TEMPERATURE

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go

config = {'displaylogo': False}

for df in dfs:
    
    df = df[['DateTime','TT2', 'TargetTemperature','TemperatureLowLimit', 'TemperatureHighLimit', 'CV1_Position']]
    df = df.dropna(axis=0, how='all')
    if df.empty:
        continue
    
    mindate = df['DateTime'].min().strftime('%Y-%m-%d')
    maxdate = df['DateTime'].max().strftime('%Y-%m-%d')

    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TT2'],
        name='Actual Temperature'
    ))
    
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TargetTemperature'],
        name='Target Temperature'
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TemperatureLowLimit'],
        name='Temperature Low Limit'
    ))
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TemperatureHighLimit'],
        name='Temperature High Limit'
    ))
    
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV1_Position'],
        name="CV1 Position",
        yaxis="y2"
    ))

    # Create axis objects
    fig.update_layout(
        xaxis=dict(domain=[0.1, 1]), #compress x axis 10% left

        yaxis=dict(
            title="Temperature"),
        yaxis2=dict(
            title="CV Position",
            anchor="x",
            overlaying="y",
            side="right"
        ),
        legend=dict(orientation="v", x = 1.1, borderwidth= 1)
    )
    

    # Update layout properties
    fig.update_layout(
        title_text=("Temperature " + mindate + " " + maxdate)
    )

    fig.show()
    fig.write_html("Temperature_" + mindate + "_" + maxdate + ".html",
                   config=config)

In [ ]:
# Flow, Alarms & Events

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.graph_objects as go

config = {'displaylogo': False}

for df in dfs:
    
    df = df[['DateTime','FT_VolumeFlow', 'FT_MassFlow', 'AlarmNumber', 'EventNumber']]
    
    mindate = df['DateTime'].min().strftime('%Y-%m-%d')
    maxdate = df['DateTime'].max().strftime('%Y-%m-%d')

    fig = go.Figure()
    
    dfGraph = df[['DateTime','FT_VolumeFlow']]
    dfGraph = dfGraph.dropna(axis=0, how='any')
    
    fig.add_trace(go.Scatter(
        x=dfGraph['DateTime'],
        y=dfGraph['FT_VolumeFlow'],
        name='Flow Transmitter VolumeFlow'
    ))
    
    if df['FT_MassFlow'].mean() != 0:
        dfGraph = df[['DateTime','FT_MassFlow']]
        dfGraph = dfGraph.dropna(axis=0, how='any')

        fig.add_trace(go.Scatter(
            x=dfGraph['DateTime'],
            y=dfGraph['FT_MassFlow'],
            name='Flow Transmitter MassFlow',
            yaxis="y2"
        ))


    dfGraph = df[['DateTime','AlarmNumber']]
    dfGraph = dfGraph.dropna(axis=0, how='any')
    
    fig.add_trace(go.Scatter(
        x=dfGraph['DateTime'],
        y=dfGraph['AlarmNumber'].astype(str),
        name='Alarm Number',
        mode='markers',marker_symbol='x',
        marker_color="red", marker_size=8,
        yaxis="y3"
    ))

    dfGraph = df[['DateTime','EventNumber']]
    dfGraph = dfGraph.dropna(axis=0, how='any')
    
    fig.add_trace(go.Scatter(
        x=dfGraph['DateTime'],
        y=dfGraph['EventNumber'].astype(str),
        name='Event Number',
        mode='markers', marker_symbol='star',
        marker_color="midnightblue",
        marker_size=8,
        yaxis="y4"
    ))

    # Create axis objects
    fig.update_layout(
        xaxis=dict(domain=[0.1, 0.9]), #compress x axis 10% left an right

        yaxis=dict(
            title="Volume Flow"),
        yaxis2=dict(
            title="Mass Flow",
            anchor="free",
            overlaying="y",
            side="left",
            position=0
        ),
        yaxis3=dict(
            title="Alarm",
            anchor="x",
            overlaying="y",
            side="right"
        ),
        yaxis4=dict(
            title="Event",
            anchor="free",
            overlaying="y",
            side="right",
            position=1
        ),
        legend=dict(orientation="v", x = 1.1, borderwidth= 1)
    )
    

    # Update layout properties
    fig.update_layout(
        title_text=("Flow, Alarms & Events " + mindate + " " + maxdate)
    )

    fig.show()
    fig.write_html("Flow-Alarms-Events_" + mindate + "_" + maxdate + ".html",
                   config=config)